Feature Selection => Backward Selection
Data Ingestion

In [1]:
from warnings import filterwarnings
filterwarnings('ignore')

In [2]:
import pandas as pd

In [ ]:
path = r"D:\Machine-Learning\repository\Data_Processing\Cars93.csv"

In [ ]:
df = pd.read_csv(path,na_values=["","NA"],keep_default_na=False)
df.head()

Perform basic data quality checks

In [ ]:
df.info()

In [ ]:
m = df.isna().sum()
m[m>0]

In [ ]:
df.duplicated().sum()

Drop insignificant columns

In [ ]:
df.drop(columns="id",inplace=True)

Separate X and Y

In [ ]:
X = df.drop(columns="Weight")
Y = df[["Weight"]]

In [ ]:
X.head()

In [ ]:
Y.head()

Data PreProcessing and Data Cleaning

In [ ]:
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer


In [ ]:
cat = list(X.columns[X.dtypes=="object"])
con = list(X.columns[X.dtypes!="object"])

In [ ]:
cat_pipe = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OrdinalEncoder()
)

In [ ]:
con_pipe = make_pipeline(
    SimpleImputer(strategy="mean"),
    StandardScaler()
)

In [ ]:
pre = ColumnTransformer([("cat",cat_pipe,cat)
                         ,("con",con_pipe,con)]).set_output(transform="pandas")

In [ ]:
pre

In [ ]:
X_pre = pre.fit_transform(X)

In [ ]:
X_pre

Feature Selection => Backward Selection

In [ ]:
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.linear_model import LinearRegression

base_model = LinearRegression()

for_sel= SequentialFeatureSelector(base_model,direction="backward",n_features_to_select=10)"auto"

for_sel.fit(X_pre,Y)

for_sel.get_feature_names_out

In [ ]:
base_model = LinearRegression()
for_sel= SequentialFeatureSelector(base_model,direction="backward",n_features_to_select="auto")

In [ ]:
for_sel.fit(X_pre,Y)

In [ ]:
imp_cols = for_sel.get_feature_names_out()
imp_cols

In [ ]:
len(imp_cols)

In [ ]:
imp_cols

In [ ]:
imp_cols[0]

In [ ]:
imp_cols[0].split("__")

In [ ]:
imp_cols[0].split("__")[1]

In [ ]:
sel_cols = [col.split("__")[1] for col in imp_cols]
sel_cols

In [ ]:
X_sel = X[sel_cols]
X_sel

Data Preprocessing : 2nd step

In [ ]:
from sklearn.preprocessing import OneHotEncoder

In [ ]:
X_sel_cat = list(X_sel.columns[X_sel.dtypes=="object"])
X_sel_con = list(X_sel.columns[X_sel.dtypes!="object"])

In [ ]:
cat_sel_pipe = make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown='ignore',sparse_output=False)
)

In [ ]:
num_sel_pipe = make_pipeline(
    SimpleImputer(strategy="mean"),
    StandardScaler()
)

In [ ]:
pre1  = ColumnTransformer([("cat",cat_sel_pipe,X_sel_cat)
                           ,("con",num_sel_pipe,X_sel_con)]).set_output(transform='pandas')

In [ ]:
pre1

In [ ]:
X_sel_pre = pre1.fit_transform(X_sel)
X_sel_pre

Train test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(X_sel_pre,Y,train_size=0.8,random_state=21)

In [ ]:
xtrain.head()

In [ ]:
xtest.head()

In [ ]:
ytrain.head()

In [ ]:
ytest.head()

Build a model

In [ ]:
model = LinearRegression()
model.fit(xtrain,ytrain)

In [ ]:
model.score(xtrain,ytrain)

In [ ]:
model.score(xtest,ytest)

Model Evaluation

In [ ]:
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score

In [ ]:
ypred = model.predict(xtrain)

In [ ]:
mse = mean_squared_error(ytrain,ypred)
mse

In [ ]:
rmse = mse**(1/2)
rmse

In [ ]:
ytrain.head()

In [ ]:
ypred[:5]

In [ ]:
ypred_test = model.predict(xtest)
ypred_test[:5]

In [ ]:
ytest.head()

We can use this model for out of sample predictions since training r2 score is around 98% and testing score is around 92%
Out of sample predctions

In [ ]:
xnew = pd.read_csv(r"D:\Machine-Learning\repository\Data_Processing\sample_cars93.csv",
                   na_values=["","NA"],keep_default_na=False)

In [ ]:
xnew.head()

In [ ]:
pre1

In [ ]:
xnew_pre = pre1.transform(xnew)
xnew_pre

In [ ]:
preds = model.predict(xnew_pre)
preds

Save the predicted results to dataframe and then to csv file

In [ ]:
xnew["WeightPredicted"] = preds
xnew

In [ ]:
xnew.to_csv("BackwardSelectionResults.csv",index=False)